In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import json


def cbf_scrap(championship, ano):
    cbf = "https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro"
    
    r = requests.get(f"{cbf}-{championship}/{ano}")
    soup = BeautifulSoup(r.text)

    slides = soup.find_all("div", {"class": "swiper-slide"})
    teams = soup.find_all("span", {"class": "time-sigla"})
    scores = soup.find_all("span", {"class": "bg-blue color-white label-2"})
    datetimes = soup.find_all("span", {"class": "partida-desc text-1 color-lightgray p-b-15 block uppercase text-center"})
    locations = soup.find_all("span", {"class": "partida-desc text-1 color-lightgray block uppercase text-center"})
    
    games = []
    for i, dtt in enumerate(datetimes):
        hscore, ascore = scores[i].text.split()[0::2]
        stadium, state = locations[i].text.split(" - ")[:2]
        game_info = {
            "dtt": datetime.strptime("T".join(dtt.text.split()[1:3]), "%d/%m/%YT%H:%M").strftime("%Y-%m-%dT%H:%M:%SZ"),
            "home": teams[i*2].text,
            "away": teams[i*2 + 1].text,
            "hscore": int(hscore),
            "ascore": int(ascore),
            "stadium": stadium.strip(),
            "state": state.strip()
        }
        games.append(game_info)
    return games

In [4]:
data = {}
championships = ["serie-a", "serie-b"]
years = range(2020, 2023)

for championship in championships:
    data[championship] = {}
    for year in years:
        data[championship][year] = cbf_scrap(championship, year)
        
data

In [ ]:
with open("cbf_data.json", "w+") as file:
    json.dump(data, file)